In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import numpy as np

### self-training 次元合わせ

In [57]:
a = torch.rand((3,240,320))     # confidence map
b = torch.rand((3, 8, 240, 320))   # before max
c, _ = torch.max(b, dim=1)    # segmentation from DeepLab after max

In [58]:
c.shape

torch.Size([3, 240, 320])

In [59]:
c[a<0.4] = -1

In [60]:
c

tensor([[[ 0.9246,  0.9158,  0.7777,  ...,  0.8583,  0.9398,  0.9359],
         [-1.0000,  0.8132,  0.9205,  ..., -1.0000,  0.7952,  0.9939],
         [-1.0000,  0.9001, -1.0000,  ..., -1.0000,  0.9548, -1.0000],
         ...,
         [ 0.5817,  0.9697, -1.0000,  ..., -1.0000,  0.9225, -1.0000],
         [ 0.9850, -1.0000, -1.0000,  ..., -1.0000,  0.8060,  0.8010],
         [ 0.9903,  0.7812, -1.0000,  ..., -1.0000, -1.0000,  0.9551]],

        [[-1.0000,  0.9994,  0.9742,  ...,  0.9942,  0.9622,  0.9956],
         [-1.0000,  0.9607,  0.9943,  ...,  0.8836, -1.0000,  0.9784],
         [ 0.9930, -1.0000,  0.8907,  ...,  0.8023, -1.0000, -1.0000],
         ...,
         [ 0.9428, -1.0000, -1.0000,  ...,  0.9413,  0.9289, -1.0000],
         [ 0.8957,  0.7855,  0.7239,  ..., -1.0000,  0.8861,  0.8826],
         [ 0.9620, -1.0000,  0.8585,  ...,  0.9540,  0.8886, -1.0000]],

        [[ 0.9821,  0.8477, -1.0000,  ...,  0.7883,  0.9274,  0.5886],
         [-1.0000, -1.0000,  0.8914,  ...,  0

# training

In [62]:
model = 1
model_D = 1

In [65]:
optimizer = optim.SGD(
                        params=[
                                        {
                                            "params": get_params(model.module, key="1x"),
                                            "lr": learning_rate,
                                            "weight_decay": CONFIG.WEIGHT_DECAY,
                                        },
                                        {
                                            "params": get_params(model.module, key="10x"),
                                            "lr": 10 * CONFIG.LR,
                                            "weight_decay": CONFIG.WEIGHT_DECAY,
                                        },
                                        {
                                            "params": get_params(model.module, key="20x"),
                                            "lr": 20 * CONFIG.LR,
                                            "weight_decay": 0.0,
                                        }],
                    momentum=CONFIG.MOMENTUM)

optimizer_D = optim.Adam(model_D.parameters(), lr=1e-4, betas=(0.9,0.99))

NameError: name 'get_params' is not defined

In [ ]:
def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter, max_iter, power):
    if iter % lr_decay_iter or iter > max_iter:
        return None
    new_lr = init_lr * (1 - float(iter) / max_iter) ** power
    optimizer.param_groups[0]["lr"] = new_lr
    optimizer.param_groups[1]["lr"] = 10 * new_lr
    optimizer.param_groups[2]["lr"] = 20 * new_lr

In [66]:
def poly_lr_scheduler_D(optimizer, init_lr, iter, lr_decay_iter, max_iter, power):
    if iter % lr_decay_iter or iter > max_iter:
        return None
    new_lr = init_lr * (1 - float(iter) / max_iter) ** power
    optimizer.param_groups[0]["lr"] = new_lr
    if len(optimizer.param_groups) > 1:
        optimizer.param_groups[1]['lr'] = 10 * new_lr

In [75]:
criterion_ce_full = nn.CrossEntropyLoss()    # weight=class_weight.to(device)
criterion_ce_semi = nn.CrossEntropyLoss(ignore_index=255)
criterion_bce = nn.BCEWithLogitsLoss()

In [67]:
# supplementary constant for discriminator
ones = torch.ones(batch_size, 240, 320).to(device)
zeros = torch.zeros(batch_size, 240, 320).to(device)

NameError: name 'batch_size' is not defined

In [ ]:
if writer_flag:
    writer = SummaryWriter('./result/')

In [79]:
def full_train(model, model_D, sample, device):
    ''' full supervised learning '''
   
    model.train()
    mode.freeze_bn()
    model_D.train()

    # train segmentation network
    batch_len = len(sample)
    x, y = sample['image'], sample['class']

    x = x.to(device)
    y = y.to(device)

    h = model(x)     # shape => (N, 8, H/8, W/8)
    h = F.interpolate(h, (240, 320), mode='bilinear')

    h_ = h.detach()    # h_ is for calculating loss for discriminator
    y_ = h.detach()    # y_is for the same purpose

    loss_ce = criterion_ce_full(h, y)
 loss_adv = criterion_bce(model_D(h), one[:batch_len]) 出力検証
    loss_full = loss_ce + loss_adv

    optimizer.zero_grad()
    optimizer_D.zero_grad()
    loss_full.backward()
    optimizer.step()


    # train discriminator
    loss_D_fake = criterion_bce(h_, zeros[:batch_len])
    loss_D_real = criterion_bce(y_, ones[:batch_len])
    loss_D = loss_D_fake + loss_D_real

    optimizer.zero_grad()
    optimizer_D.zero_grad()
    loss_D.backward()
    optimizer_D.step()

    return loss_full.item(), loss_D.item()

In [ ]:
def semi_train(model, model_D, sample, device):
    """ semi-supervised learning. Discriminator is not trained. """

    # train segmentation network
    batch_len = len(sample)
    x = sample['image']
    x = x.to(device)

    h = model(x)     # shape => (N, 8, H/8, W/8)
    h = F.interpolate(h, (240, 320), mode='bilinear')
    
    out = model_D(h)    # shape => (N, W, H)
    
 loss_adv = criterion_bce(model_D(h), one[:batch_len])    出力検証 one-hot の逆にしなきゃいけない？

    
    optimizer.zero_grad()
    optimizer_D.zero_grad()
    loss_D.backward()
    optimizer_D.step()

    return loss_full.item(), loss_D.item()

In [ ]:
losses_full = []
losses_semi =[]
losses_D = []
class_IOU = []

for epoch in range(max_epoch=200):
    
    model.train()
    mode.freeze_bn()
    model_D.train()
    
    epoch_loss_full = 0.0
    epoch_loss_D = 0.0
    epoch_loss_semi = 0.0
    
    poly_lr_scheduler(
        optimizer=optimizer,
        init_lr=CONFIG.LR,
        iter=iteration - 1,
        lr_decay_iter=CONFIG.LR_DECAY,
        max_iter=CONFIG.ITER_MAX,
        power=CONFIG.POLY_POWER,
    )

    poly_lr_scheduler(
        optimizer=optimizer,
        init_lr=CONFIG.LR,
        iter=iteration - 1,
        lr_decay_iter=CONFIG.LR_DECAY,
        max_iter=CONFIG.ITER_MAX,
        power=CONFIG.POLY_POWER,
    )
    
    
    # only supervised learning
    if epoch < 5:
        for i, sample in enumerate(train_loader_with_label):
            
            loss_full, loss_D = full_train(model, model_D, sample, device)
            
            epoch_loss_full += loss_full
            epoch_loss_D += loss_D
            
        losses_full.append(epoch_loss_full / i)   # mean loss over all samples
        losses_full.append(epoch_loss_D / i)
        
    
    # semi-supervised learning
    if epoch >= 5:
        for i, (sample1, sample2) in enumerate(zip(train_loader_with_label, train_loader_without_label)):
            
            
        
        

3